In [1]:
import pyodbc
import datetime
from datetime import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


ip_date = "25-02-2022"
day, month, year = map(int, ip_date.split('-'))
ip_date = datetime.datetime(year, month, day)
ip_start_time = "08:32:00"
h, m, s = map(int, ip_start_time.split(':'))
ip_start_time = datetime.datetime(year, month, day, h, m, s)
print(ip_start_time)
ip_end_time = "09:04:00"
eh, em, es = map(int, ip_end_time.split(':'))
ip_end_time = datetime.datetime(year, month, day, eh, em, es)
dvalue_code1 = 0.92
dvalue_code2 = 0.97
dvalue_code3 = 0.90
dvalue_code4 = 0.81
dvalue_code5 = 0.8
dvalue_code6 = 0.95
d_values = [dvalue_code1, dvalue_code2, dvalue_code3,
            dvalue_code4, dvalue_code5, dvalue_code6]
tolerance = 0.01

2022-02-25 08:32:00


In [2]:
connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=GOVIND\MSSQLSERVER01;DATABASE=backtest;Trusted_Connection=yes;')
query = """select * from dbo.Day3;"""
df = pd.read_sql(query, connection)


In [3]:
df['start_date_time'] = df.apply(lambda r : pd.datetime.combine(r['Date'],r['Time']),1)
df = df.drop(['Date', 'Time'], axis = 1)
df = df.dropna()
df.head()

,Code,Dvalue,Price,FinalDate,start_date_time
0,Code4100C,0.93,196.45,2022-02-25,2022-02-25 08:31:21
1,Code4105C,0.93,191.50,2022-02-25,2022-02-25 08:31:21
2,Code4110C,0.92,186.55,2022-02-25,2022-02-25 08:31:21
3,Code4115C,0.92,181.60,2022-02-25,2022-02-25 08:31:21
4,Code4120C,0.92,176.65,2022-02-25,2022-02-25 08:31:21


In [4]:
df = df[df.start_date_time.between(ip_start_time, ip_end_time)]
if len(df) == 0:
    raise "No records found between the given times"
unique_final_dates = df['FinalDate'].unique()
final_date_dict = {}

for final_date in unique_final_dates:
    final_date_dict[final_date] = {"data":df[df['FinalDate'] == final_date]}
    print(final_date_dict[final_date]["data"].head())

           Code  Dvalue  Price   FinalDate     start_date_time
1023  Code4310P   -0.56  47.70  2022-02-28 2022-02-25 08:32:00
1024  Code4315P   -0.58  50.25  2022-02-28 2022-02-25 08:32:00
1025  Code4320P   -0.60  53.05  2022-02-28 2022-02-25 08:32:00
1026  Code4325P   -0.62  55.90  2022-02-28 2022-02-25 08:32:01
1027  Code4330P   -0.64  59.45  2022-02-28 2022-02-25 08:32:01
           Code  Dvalue   Price   FinalDate     start_date_time
1062  Code4100C     1.0  194.90  2022-02-25 2022-02-25 08:32:06
1063  Code4105C     1.0  190.00  2022-02-25 2022-02-25 08:32:07
1064  Code4110C     1.0  185.00  2022-02-25 2022-02-25 08:32:07
1065  Code4115C     1.0  180.10  2022-02-25 2022-02-25 08:32:07
1066  Code4120C     1.0  175.15  2022-02-25 2022-02-25 08:32:07


In [13]:
for key_date in final_date_dict.keys():
    temp = final_date_dict[key_date]['data']
    df_dval_filtered = pd.DataFrame([], columns=list(temp.columns))
    print(df_dval_filtered)
    avail_dvals = list(temp['Dvalue'].unique())
    dvalue_first = pd.DataFrame([], columns=list(temp.columns))
    for d_value in d_values:
        curr_dvalue = d_value
        df_dvals = temp[temp['Dvalue'] == curr_dvalue]
        while len(df_dvals) == 0:
            # print(len(df_dvals), d_value)
            less_curr_dvalue, high_curr_dvalue = curr_dvalue - tolerance, curr_dvalue + tolerance
            if less_curr_dvalue in avail_dvals:
                curr_dvalue = less_curr_dvalue
                df_dvals = temp[temp['Dvalue'] == curr_dvalue]
            elif high_curr_dvalue in avail_dvals:
                curr_dvalue = high_curr_dvalue
                df_dvals = temp[temp['Dvalue'] == curr_dvalue]
        #print(df_dvals.iloc[0])
        dvalue_first = pd.concat([dvalue_first, df_dvals.head(1)], axis=0, ignore_index=True)
        #df_dval_filtered = pd.concat([df_dval_filtered, df_dvals], axis=0, ignore_index=True)
        # final_date_dict_filtered[]
        #final_date_dict[key_date]["filtered"] = df_dval_filtered
        final_date_dict[key_date]["first"] = dvalue_first

Empty DataFrame
Columns: [Code, Dvalue, Price, FinalDate, start_date_time]
Index: []
Empty DataFrame
Columns: [Code, Dvalue, Price, FinalDate, start_date_time]
Index: []


In [6]:
for date, val in final_date_dict.items():
    code_list = val['first']['Code'].to_list()
    last_codes = final_date_dict[date]['data'].groupby(['Code'], as_index=False).last()
    last_codes = last_codes[last_codes['Code'].isin(code_list[:-2])]
    pre_step1_df = val['first'].merge(last_codes, on='Code', how='left')
    pre_step1_df =  pre_step1_df[["Code", "Dvalue_x", "FinalDate_x", "start_date_time_x", "Price_x", "Price_y"]].dropna()
    pre_step1_df.rename(columns = {
        'Dvalue_x':'Dvalue', 
        'FinalDate_x':'FinalDate', 
        'start_date_time_x':'StartDateTime',
        'Price_x':'IntialPrice',
        'Price_y':'CurrentPrice'
        }, inplace = True)
    pre_step1_df['PnL'] = pre_step1_df['CurrentPrice'] - pre_step1_df['IntialPrice']
    final_date_dict[date]['current'] = pre_step1_df
    #pre_step1_df['StartTime'] = ip_start_time
    pre_step1_df['EndTime'] = ip_end_time
    print(pre_step1_df)

    

    
#add end time
    

        Code Dvalue   FinalDate        StartDateTime IntialPrice  \
0  Code4115C   0.92  2022-02-28  2022-02-25 08:32:15      184.35   
1  Code4080C   0.97  2022-02-28  2022-02-25 08:32:55      221.65   
2  Code4135C    0.9  2022-02-28  2022-02-25 08:32:15      165.45   
3  Code4195C   0.81  2022-02-28  2022-02-25 08:32:16       111.2   

   CurrentPrice   PnL             EndTime  
0        179.90 -4.45 2022-02-25 09:04:00  
1        214.05  -7.6 2022-02-25 09:04:00  
2        161.25  -4.2 2022-02-25 09:04:00  
3        109.40  -1.8 2022-02-25 09:04:00  
        Code Dvalue   FinalDate        StartDateTime IntialPrice  \
0  Code4130C   0.92  2022-02-25  2022-02-25 08:32:37       171.7   
1  Code4100C   0.97  2022-02-25  2022-02-25 08:32:37      200.65   
2  Code4175C    0.9  2022-02-25  2022-02-25 08:32:07       122.3   
3  Code4215C   0.81  2022-02-25  2022-02-25 08:32:38        89.5   

   CurrentPrice   PnL             EndTime  
0        163.35 -8.35 2022-02-25 09:04:00  
1        1

In [7]:
query = """select * from dbo.vc_vp_constants;"""
const_vals_df = pd.read_sql(query, connection)

In [8]:
vc1_const = float(const_vals_df['vc1_constant'])
vc2_const = float(const_vals_df['vc2_constant'])
vc11_const = float(const_vals_df['vc11_constant'])
vc21_const = float(const_vals_df['vc21_constant'])
vp1_const = float(const_vals_df['vp1_constant'])
vp2_const = float(const_vals_df['vp2_constant'])
vp11_const = float(const_vals_df['vp11_constant'])
vp21_const = float(const_vals_df['vp21_constant'])


In [100]:
for date, val in final_date_dict.items():
    print("Date", date)
    Code1_val = val['current']['Code'][0]
    Code2_val = val['current']['Code'][1]
    Code5_val = val['first']['Code'][4]
    Code6_val = val['first']['Code'][5]
    codes_list = [Code1_val, Code2_val]
    VC1 = val['current']['IntialPrice'][0] + val['current']['IntialPrice'][1] + vc1_const
    VC2 = VC1 + vc2_const
    VC11 = VC1 - vc11_const
    VC21 = VC2 - vc21_const
    VP1 = val['current']['IntialPrice'][0] + val['current']['IntialPrice'][1] - vp1_const
    VP2 = VP1 + vp2_const
    VP11 = VP1 - vp11_const 
    VP21 = VP2 - vp21_const
    df['FinalDate'] = pd.to_datetime(df['FinalDate'], format='%Y-%m-%d')
    VC1 = 180
    VC2 = 180
    VC11 = 180
    VC21 = 180
    
    VP1 = 180
    VP2 = 180
    VP11 = 180
    VP21 = 180
    #print(f'VC1 = {VC1}, VC2 = {VC2}, VC11 = {VC11}, VC21 = {VC21}, VP1 = {VP1}, VP2 = {VP2}, VP11 = {VP11}, VP21 = {VP21}')
    
    code1n2_filtered = final_date_dict[date]['data'][(final_date_dict[date]['data']['Code'] == Code1_val) | (final_date_dict[date]['data']['Code'] == Code2_val)]
    code5n6_filtered = final_date_dict[date]['data'][(final_date_dict[date]['data']['Code'] == Code5_val) | (final_date_dict[date]['data']['Code'] == Code6_val)]
    con1 = False
    con3 = False
    con5 = False
    con7 = False
    portfolio_df = pd.DataFrame(columns=['Code', 'StartDate', 'FinalDate', 'InitialPrice', 'Price', 'TriggerType'])
    code5n6_filtered['start_date_time'] = pd.to_datetime(code5n6_filtered['start_date_time'])

    for index, row in code1n2_filtered.iterrows():
        if row['Code'] == Code1_val:
            if (row['Price'] > VC1) and not con1:
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                con1 = True
                code5_row = code5n6_filtered[code5n6_filtered['Code'] == Code5_val].head(1)
                print(code5_row)
                if len(code5_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code5_row['Price'], 'Price':row['Price'], 'TriggerType':'Buy'}
                
            if (row['Price'] > VC2) and not con3:
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                con3 = True
                code5_row = code5n6_filtered[code5n6_filtered['Code']==Code5_val].head(1)
                print(code5_row)
                if len(code5_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code5_row['Price'], 'Price':row['Price'], 'TriggerType':'Buy'}
                
            if (row['Price'] < VC11) and (con1 or con3):
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                code5_row = code5n6_filtered[code5n6_filtered['Code']==Code5_val].head(1)
                print(code5_row)
                
                if len(code5_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code5_row['Price'], 'Price':row['Price'], 'TriggerType':'Sell'}
                
                if con1:
                    con1 = False
                elif con3:
                    con3 = False
            if (row['Price'] < VC21) and (con1 or con3):
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                code5_row = code5n6_filtered[code5n6_filtered['Code']==Code5_val].head(1)
                print(code5_row)
                
                if len(code5_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code5_row['Price'], 'Price':row['Price'], 'TriggerType':'Sell'}
                if con1:
                    con1 = False
                elif con3:
                    con3 = False
                
        elif row['Code'] == Code2_val:
            if (row['Price'] > VP1) and not con5:
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                con5 = True
                code6_row = code5n6_filtered[code5n6_filtered['Code']==Code6_val].head(1)
                print(code6_row)
                
                if len(code6_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code6_row['Price'], 'Price':row['Price'], 'TriggerType':'Buy'}
            if (row['Price'] > VP2) and not con7:
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                con7 = True
                code6_row = code5n6_filtered[code5n6_filtered['Code']==Code6_val].head(1)
                print(code6_row)
                
                if len(code6_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code6_row['Price'], 'Price':row['Price'], 'TriggerType':'Buy'}
            if (row['Price'] < VP11) and (con5 or con7):
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                code6_row = code5n6_filtered[code5n6_filtered['Code']==Code6_val].head(1)
                print(code6_row)
                
                if len(code6_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code6_row['Price'], 'Price':row['Price'], 'TriggerType':'Sell'}
                if con5:
                    con5 = False
                elif con7:
                    con7 = False
                    
            if (row['Price'] < VP21) and (con5 or con7):
                portfolio_df = portfolio_df.append(new_row, ignore_index=True)
                code6_row = code5n6_filtered[code5n6_filtered['Code']==Code6_val].head(1)
                print(code6_row)
                
                if len(code6_row) == 0:
                    raise "No Rows Found in code5n6_filtered with start_date_time greater than " + str(row['start_date_time'])
                new_row = {'Code':row['Code'], 'StartDate':row['start_date_time'], 'FinalDate': row['FinalDate'],'InitialPrice':code6_row['Price'], 'Price':row['Price'], 'TriggerType':'Sell'}
                
                if con5:
                    con5 = False
                elif con7:
                    con7 = False
    final_date_dict[date]['log'] = portfolio_df
    print(portfolio_df.head())    

Date 2022-02-28
           Code  Dvalue  Price   FinalDate     start_date_time
1216  Code4075C    0.95  222.8  2022-02-28 2022-02-25 08:32:15
           Code  Dvalue  Price   FinalDate     start_date_time
1216  Code4075C    0.95  222.8  2022-02-28 2022-02-25 08:32:15
           Code  Dvalue   Price   FinalDate     start_date_time
1241  Code4200C     0.8  107.95  2022-02-28 2022-02-25 08:32:16
           Code  Dvalue   Price   FinalDate     start_date_time
1241  Code4200C     0.8  107.95  2022-02-28 2022-02-25 08:32:16
           Code  Dvalue   Price   FinalDate     start_date_time
1241  Code4200C     0.8  107.95  2022-02-28 2022-02-25 08:32:16
           Code  Dvalue   Price   FinalDate     start_date_time
1241  Code4200C     0.8  107.95  2022-02-28 2022-02-25 08:32:16
           Code  Dvalue   Price   FinalDate     start_date_time
1241  Code4200C     0.8  107.95  2022-02-28 2022-02-25 08:32:16
           Code  Dvalue   Price   FinalDate     start_date_time
1241  Code4200C     0.8  107